# Word (.docx) to CSV

In [1]:
from translation_helpers import docx_to_csv

In [2]:
word_file = "../../data/Parkinson/en/7-1_script_interview_clinique_1 English.docx"

docx_to_csv(word_file, data_directory="../../data")
# To save in the main results directory
#docx_to_csv(word_file, output_directory="../../results", data_directory="../../data")

'results\\Parkinson\\en\\7-1_script_interview_clinique_1 English.csv'

# Translation

In [3]:
from transformers import SeamlessM4Tv2ForTextToText, AutoProcessor
import torch
from translation_helpers import translate_csv

c:\Users\david\anaconda3\envs\seam\lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

True
1
NVIDIA GeForce GTX 1050 Ti


In [5]:
model = SeamlessM4Tv2ForTextToText.from_pretrained("facebook/seamless-m4t-v2-large")
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
cuda = True

if cuda and torch.cuda.is_available():
    model = model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Check the max position embeddings (equivalent to max token length)
max_length = model.config.max_position_embeddings
print(f"Max token length: {max_length}")

Max token length: 4096


In [7]:
text = "Your input text goes here."
tokens = processor(text, return_tensors="pt")

print(f"Number of tokens: {len(tokens['input_ids'][0])}")

Number of tokens: 8


### Translation CSV

In [ ]:
source_lang = "fra" # French
target_lang = "eng" # English
path_file = "csv/7-1_script_interview_clinique_4_21-08-2020.csv"

translate_csv(path_file, source_lang, target_lang, model, processor, cuda)